In [1]:
print_imports()

Loaded modules:
numpy                np              1.14.0
pandas               pd              0.22.0
sklearn              sk              0.19.1
keras                ks              2.1.4

matplotlib           mpl             2.1.2
matplotlib.pyplot    plt             N/A
matplotlib.image     mpimg           N/A
seaborn              sns             0.8.1
PIL                  PIL             5.0.0

ExergyUtilities      exergy          2.0.

pyspark              pyspark         2.2.1


### Imports

In [2]:
import os, re, logging, json
from pprint import pprint
from keras import backend as K

In [3]:
mod_path = r"/home/batman/git/ai_drive/src"
sys.path.append(mod_path)
logging.debug("ADDED TO PATH: ".format(mod_path))

In [4]:
import drive.analysis_offline as analysis

### Paths

In [5]:
this_project_path = r"/media/batman/USB STICK"
project_name = r'catdog1'

### Plotting function

In [6]:
model_title = "10 Epochs"
def plot_hist(history_dict):
    #fig = plt.figure(figsize=(5,4))
    #fig=plt.figure(figsize=(20, 10),facecolor='white')

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5),sharey=False,facecolor='white')
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5),sharey=False,facecolor='0.15')
    
    ax1.plot(history_dict['epoch'],  history_dict['history']['loss'],label="Train")
    ax1.plot(history_dict['epoch'],  history_dict['history']['val_loss'],label="CV")
    ax1.set_title("Loss function development - Training set vs CV set")
    ax1.legend(loc='upper right')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Values')
    
    ax2.plot(history_dict['epoch'],  history_dict['history']['acc'],label="Train")
    ax2.plot(history_dict['epoch'],  history_dict['history']['val_acc'],label="CV")
    ax2.set_title("Accuracy development - Training set vs CV set")
    ax2.legend(loc='upper right')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Values')
    
    plt.suptitle(model_title, fontsize=16)
    
    plt.show()

#plot_hist(history_dict)

```total_params = (filter_height * filter_width * input_image_channels + 1) * number_of_filters```

### Each loaded layer has a summary string

In [10]:
def Conv2D(params):
    #print(params['class_name'])
    #pprint(params['config'])
    kernel_dim = (params['config']['kernel_size'][0])
    filters = (params['config']['filters'])
    return "{}, kernel {}, filters {}".format(params['class_name'],kernel_dim,filters)

def MaxPooling2D(params):
    #print(params['class_name'])    
    #pprint(params['config'])
    pool_size = (params['config']['pool_size'][0])
    
    return "{}, pool {}".format(params['class_name'],pool_size)
def Flatten(params):
    #print(params['class_name'])
    return "{}".format(params['class_name'])

def Dropout(params):
    #print(params['class_name'])
    #pprint(params['config'])
    drp_rate = (params['config']['rate'])
    return "{}, dropout {}".format(params['class_name'],drp_rate)
    
    
    #raise
def Dense(params):
    #print(params['class_name'])
    return "{}".format(params['class_name'])

layer_funcs = {
        'Conv2D':Conv2D,
        'MaxPooling2D':MaxPooling2D,
        'Flatten':Flatten,
        'Dropout':Dropout,
        'Dense':Dense,
}

### Main

In [11]:
def count_params(model):
    trainable_count = int(
        np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(
        np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))
    return {'Total':trainable_count + non_trainable_count,'Trainable':trainable_count,'Non-trainable':non_trainable_count}
    #print('Total params: {:,}'.format(trainable_count + non_trainable_count))
    #print('Trainable params: {:,}'.format(trainable_count))
    #print('Non-trainable params: {:,}'.format(non_trainable_count))

In [17]:
root_path = os.path.join(this_project_path,project_name)
run_folders = [dir for dir in os.listdir(root_path) if re.match('run',dir)]
run_folders.sort()

# Loop run folders
for rfolder in run_folders:
    this_run_path = os.path.join(root_path,rfolder)
    logging.debug('**** RUN {} ****'.format(rfolder))
    
    ###### Weights ######
    wts = analysis.get_weights(this_run_path)
    if wts: best_wt = wts[-1] # BEST weight (last weight)

    ###### Architecture ######
    model = analysis.load_model(this_run_path)
    arch_dict = analysis.read_model_json(this_run_path)
    
    #model.summary()
    print(count_params(model))
    
    ##### Reload weights #####
    if wts:
        print(best_wt)
        model.load_weights(best_wt['path'])
    #raise
    ###### Loop layers ######
    for i,layer in enumerate(arch_dict['config']):
        layer_str = layer_funcs[layer['class_name']](layer)
        print(i,layer_str)

    ###### History ######
    path_hist = analysis.get_history(this_run_path)    
    with open(path_hist) as hist_file:
        hist_dict = json.load(hist_file)
    print("Epochs",hist_dict['params']['epochs'])
    print("Steps",hist_dict['params']['steps'])
    
    #plot_hist(hist_dict)
    del model
    raise


root  - 10  - <ipython-input-17-4579f88942c5>  <module>                      : **** RUN run078 ****
root  - 10  - analysis_offline      get_weights                   : Found 25 weights files, total 660 MB = 26.4 MB per file
{'Total': 3453121, 'Trainable': 3453121, 'Non-trainable': 0}
{'epoch': 50, 'fname': 'weights-epoch50-0.91.hdf5', 'path': '/media/batman/USB STICK/catdog1/run078/weights-epoch50-0.91.hdf5', 'size': 26.384925842285156}
0 Conv2D, kernel 3, filters 32
1 MaxPooling2D, pool 2
2 Conv2D, kernel 3, filters 64
3 MaxPooling2D, pool 2
4 Conv2D, kernel 3, filters 128
5 MaxPooling2D, pool 2
6 Conv2D, kernel 3, filters 128
7 MaxPooling2D, pool 2
8 Flatten
9 Dropout, dropout 0.5
10 Dense
11 Dense
/media/batman/USB STICK/catdog1/run078/history run078.txt
Epochs 50
Steps 400.0


RuntimeError: No active exception to reraise

NameError: name 'model' is not defined